In [ ]:
### Import Functions
from DirectFunctions import import_fx_data, direct_daily_rets, get_pairs, compute_hedge_ratio, calculate_hedged_unhedged
import pandas as pd


In [ ]:
### Call Function to Load in Rate Data
all_forex_pairs = [
    'EURUSD=X', 'GBPUSD=X', 'USDJPY=X', 'USDCHF=X', 'AUDUSD=X', 'NZDUSD=X',
    'USDCAD=X', 'EURGBP=X', 'EURJPY=X', 'EURCHF=X', 'EURAUD=X', 'EURNZD=X',
    'EURCAD=X', 'GBPAUD=X', 'GBPJPY=X', 'GBPCHF=X', 'AUDJPY=X', 'AUDCHF=X',
    'AUDNZD=X', 'NZDJPY=X', 'NZDCHF=X', 'CADJPY=X', 'CADCHF=X', 'CHFJPY=X'
]
start_date = '2023-07-01'
end_date = '2024-07-01'
forex_data = import_fx_data(all_forex_pairs, start_date, end_date)
forex_data.tail()


In [ ]:
### Call Function to Calculate Daily Returns
daily_returns = direct_daily_rets(forex_data)
print(daily_returns.head())


In [ ]:
### Function Call
all_pairs = get_pairs(all_forex_pairs)
print(f'Total pairs: {len(all_pairs)}')  # Should print 552

In [ ]:
### Append Long Position, Short Position, Hedged returns, and Unhedged Returns Columns
# Create an empty DataFrame to store hedge ratios
hedge_ratios_df = pd.DataFrame(columns = ['Pair1', 'Pair2', 'Hedge Ratio', 'Long Position', 'Short Position', 'Hedged Returns', 'Unhedged Returns'])
long_pos = 1000
# Loop over pair combinations to compute hedge ratio and returns
for pairing in all_pairs:
    try:
        hedge_ratio = compute_hedge_ratio(pairing, daily_returns)
        short_pos, hedged_cumulative_rets, unhedged_cumulative_rets = calculate_hedged_unhedged(long_pos, pairing, daily_returns, hedge_ratio)
        new_row = pd.DataFrame({
            'Pair1': [pairing[0]], 
            'Pair2': [pairing[1]], 
            'Hedge Ratio': [hedge_ratio], 
            'Long Position': [long_pos], 
            'Short Position': [short_pos],
            'Hedged Returns': [hedged_cumulative_rets.iloc[-1]], 
            'Unhedged Returns': [unhedged_cumulative_rets.iloc[-1]]
        })
        hedge_ratios_df = pd.concat([hedge_ratios_df, new_row], ignore_index=True)
    except Exception as e:
        print(f"Error computing hedge ratio for {pairing}: {e}")

# Display the DataFrame
print(hedge_ratios_df.tail())


In [ ]:
### Filter for Most Profitable Hedged Pairing
# Sort the DataFrame by the 'Hedged Returns' column in descending order
most_profit_hedged = hedge_ratios_df.sort_values(by = 'Hedged Returns', ascending = False).head(1)
#print(most_profit_hedged)

### Display Results
pair1 = most_profit_hedged.iloc[0]['Pair1']
pair2 = most_profit_hedged.iloc[0]['Pair2']
hedge_ratio = most_profit_hedged.iloc[0]['Hedge Ratio']
long_position = most_profit_hedged.iloc[0]['Long Position']
short_position = most_profit_hedged.iloc[0]['Short Position']
hedged_returns = most_profit_hedged.iloc[0]['Hedged Returns']
unhedged_returns = most_profit_hedged.iloc[0]['Unhedged Returns']

print(f"Most Profitable Pairings: {pair1} and {pair2}")
print(f"Hedge Ratio: {hedge_ratio:.6f}")
print(f"Long Position: {long_position}")
print(f"Short Position: {short_position:.6f}")
print("\nReturns:")
print(f"Hedged Returns: {hedged_returns:.6f}")
print(f"Unhedged Returns: {unhedged_returns:.6f}")
